In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import torch.nn
import torchtext
from torchtext import data
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
import torch
from torch.utils.data import TensorDataset, DataLoader

import pandas as pd
import numpy as np
import timeit

import spacy

In [0]:
PATH = "/content/drive/My Drive/Colab Notebooks/movieReccomendation/dataset/final_data/"

In [5]:
X = pd.read_csv(PATH+'train.csv')
print(X.head())

print(X.shape)

   sentiment                                       clean_review
0          0  history year movie critic lead general public ...
1          0  love movie admit well straight video movie see...
2          1  think movie laugh anne ramsey play unforgettab...
3          1  teen set camp oregon wilderness despite warn p...
4          1  affable aspire cartoonist hoop mccann wonderfu...
(40000, 2)


In [6]:
X.clean_review[0]

'history year movie critic lead general public astray send eye wide shut hype blair witch project magnolia far wrong movie year heck know self indulgence load bad act like stay watch credit thing movie theatre fast audience attend show feel pretty way date argue stranger ugly hateful film feel ugly hateful'

In [7]:
def load_file(filepath, device):
    tokenizer = lambda x: str(x).split()

    TEXT = data.Field(include_lengths = True)
    LABEL = data.Field(sequential=False, use_vocab=False, is_target=True)
    tv_datafields = [('sentiment', LABEL), ('clean_review', TEXT)]
    # Step two construction our dataset.
    train, valid, test = data.TabularDataset.splits(path=filepath,
                                                    train='train.csv', validation='val.csv',
                                                    test='test.csv', format="csv",
                                                    skip_header=True, fields=tv_datafields)
    print(train[1].__dict__.keys())
    # Step three We should build_vocab for the field with use_vocab=True. 
    # If not we will get an error during the loop section.
    TEXT.build_vocab(train, 
                     vectors = "glove.6B.100d", 
                     unk_init = torch.Tensor.normal_)
    LABEL.build_vocab(train)
    print("build vocab success...")

    # Step four construct our iterator to our dataset. 
    train_iter, valid_iter, test_iter = data.BucketIterator.splits( 
                                    (train, valid, test), batch_sizes=(64, 8, 8),
                                    sort_key=lambda x: len(x.clean_review), device=device)
    print("construct iterator success...")

    return TEXT, LABEL, train, valid, test, train_iter, valid_iter, test_iter

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
TEXT, LABEL, train, valid, test, train_iter, valid_iter, test_iter = load_file(PATH, device)

dict_keys(['sentiment', 'clean_review'])
build vocab success...
construct iterator success...


In [8]:
mydict = train[0].__dict__
#lambda x: len(x.clean_review)
print(mydict['clean_review'])
print(dir(train))
print(train[0])

['history', 'year', 'movie', 'critic', 'lead', 'general', 'public', 'astray', 'send', 'eye', 'wide', 'shut', 'hype', 'blair', 'witch', 'project', 'magnolia', 'far', 'wrong', 'movie', 'year', 'heck', 'know', 'self', 'indulgence', 'load', 'bad', 'act', 'like', 'stay', 'watch', 'credit', 'thing', 'movie', 'theatre', 'fast', 'audience', 'attend', 'show', 'feel', 'pretty', 'way', 'date', 'argue', 'stranger', 'ugly', 'hateful', 'film', 'feel', 'ugly', 'hateful']
['__add__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'download', 'examples', 'fields', 'filter_examples', 'sort_key', 'split', 'splits']


In [9]:
# most common words and their frequencies.
print(TEXT.vocab.freqs.most_common(20))

# top ten index to words transform.
print(TEXT.vocab.itos[:10])

[('movie', 80774), ('film', 74742), ('like', 34849), ('time', 24701), ('good', 23395), ('character', 22260), ('watch', 21587), ('story', 19637), ('see', 19625), ('think', 19329), ('well', 19018), ('scene', 16640), ('great', 15855), ('look', 15711), ('know', 15233), ('end', 14575), ('bad', 14451), ('people', 14447), ('go', 14312), ('get', 13922)]
['<unk>', '<pad>', 'movie', 'film', 'like', 'time', 'good', 'character', 'watch', 'story']


In [10]:
for i,batch in enumerate(valid_iter):
    print(i)
    if(i==0):
        print('Batch:', batch)
        print('batch.clean_review: \n', batch.clean_review)
        print('batch.sentiment : \n', batch.sentiment)
        print('batch.clean_review.size :', batch.clean_review[0].size())
        print('batch.sentiment.size: ', batch.sentiment.size())
        break

0
Batch: 
[torchtext.data.batch.Batch of size 8]
	[.sentiment]:[torch.cuda.LongTensor of size 8 (GPU 0)]
	[.clean_review]:('[torch.cuda.LongTensor of size 12x8 (GPU 0)]', '[torch.cuda.LongTensor of size 8 (GPU 0)]')
batch.clean_review: 
 (tensor([[   21,  1643,    58,   360,  5921,     7,  5940,    12],
        [   96,   583,   299,  1449,    10,  4472, 22950,   562],
        [    2,   458,     3,  1829,     4,    90,   180,   612],
        [  557,     2,   150,  2263,   329,   243,     3,  4705],
        [  655,     8,     8,  2318,   815,   155,    30,     8],
        [  647,   348,    14,  2632,     6,   354,   920,     1],
        [   85,    62,    24, 32374,    32,  6234,    52,     1],
        [  150,   486,    49,   151,   378, 14634,     1,     1],
        [    8,   100,     6,    70,    24,     1,     1,     1],
        [   11,    12,     3,     8, 48314,     1,     1,     1],
        [ 6474,   322,   134,     1,     1,     1,     1,     1],
        [    6,     1,     1,     1

In [0]:
class SentimentClassification(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, n_layers, pad_idx, dropout):
        super(SentimentClassification, self).__init__()

        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx = pad_idx)
        
        self.lstm = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=True, 
                           dropout=dropout, batch_first=True)
        
        self.dropout = nn.Dropout(dropout)

        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        self.sig = nn.Sigmoid()

    def forward(self, text, text_lengths):
        batch_size = text.size(1)       
        #text_dim = (seq_len, batch_size)
        
        #embed_dim = (seq_len, batch_size, embed_dim)
        embeds = self.embedding(text)
        
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embeds, text_lengths, enforce_sorted=False)
        packed_output, (hidden, cell) = self.lstm(packed_embedded)

        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
        #print('out dim ', output.size())
        #output = [sent len, batch size, hid dim * num directions]
        # stack up lstm outputs
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim] 
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        out = self.fc(hidden)
        #hidden = [batch size, hid dim * num directions] 
        return out

    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (torch.cuda.is_available()):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden
            

In [12]:
INPUT_DIM = len(TEXT.vocab)
print(INPUT_DIM)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
DROPOUT = 0.3
model = SentimentClassification(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS, PAD_IDX, DROPOUT)
print(model)

141208
SentimentClassification(
  (embedding): Embedding(141208, 100, padding_idx=1)
  (lstm): LSTM(100, 256, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=512, out_features=1, bias=True)
  (sig): Sigmoid()
)


In [0]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.BCEWithLogitsLoss()
model = model.to(device)
criterion = criterion.to(device)

def calculateAccuracy(preds, y):
    '''
    Return accuracy per batch ..
    '''
    # round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    
    return acc

In [14]:
for i,batch in enumerate(train_iter):
    if(i==0):
        print('Batch:', batch)
        print('batch.clean_review: \n', batch.clean_review)
        print('batch.sentiment : \n', batch.sentiment)
        print('batch.clean_review.size :', batch.clean_review[0].size())
        print('batch.sentiment.size: ', batch.sentiment.size())
        break

Batch: 
[torchtext.data.batch.Batch of size 64]
	[.sentiment]:[torch.cuda.LongTensor of size 64 (GPU 0)]
	[.clean_review]:('[torch.cuda.LongTensor of size 425x64 (GPU 0)]', '[torch.cuda.LongTensor of size 64 (GPU 0)]')
batch.clean_review: 
 (tensor([[ 252,  593,   25,  ...,   35,   34,  847],
        [   3, 1987,  201,  ...,  260,  511,   12],
        [  22,  342,   70,  ..., 1956, 1071,  493],
        ...,
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1]], device='cuda:0'), tensor([ 66,  21, 171, 269,  67,  68,  99,  74,  79,  52, 120,  76,  41, 184,
         84, 139,  72,  98, 425,  35,  61,  51,  55, 422,  52,  97,  96,  79,
         52, 143, 218, 331,  44, 123,  66,  69, 110,  25,  94, 121, 104,  49,
        110, 170,  50,  64,  91,  58,  63,  69, 207, 123,  92, 171, 210, 118,
         34, 193,  52,  76,  19,  46, 199, 165], device='cuda:0'))
batch.sentiment : 
 tensor([1, 0, 0

In [17]:
MODEL_PATH = "/content/drive/My Drive/Colab Notebooks/movieReccomendation/"
epochs = 1 # 3-4 is approx where I noticed the validation loss stop decreasing
best_val_loss = float('inf')
clip=5 # gradient clipping
# move model to GPU, if available
if(torch.cuda.is_available()):
        model.cuda()
# train for some number of epochs
for e in range(epochs):
    losses = []
    train_acc = []
    model.train()
    # batch loop
    for i, batch in enumerate(train_iter):
        
        # zero accumulated gradients
        optimizer.zero_grad()
        #print('text', batch.clean_review.size())
        # get the output from the model
        predictions = model(batch.clean_review[0], batch.clean_review[1]).squeeze(1)

        # calculate the loss and perform backprop
        loss = criterion(predictions, batch.sentiment.float())
        loss.backward()
        acc = calculateAccuracy(predictions, batch.sentiment)
        losses.append(loss.item())
        train_acc.append(acc.item())
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        #print(loss.item())

    val_losses = []
    val_accs = []
    model.eval()
    for i_val, batch_val in enumerate(valid_iter):

        val_pred = model(batch_val.clean_review[0], batch_val.clean_review[1]).squeeze(1)
        val_loss = criterion(val_pred, batch_val.sentiment.float())
        val_acc = calculateAccuracy(val_pred, batch_val.sentiment)
        val_losses.append(val_loss.item())
        val_accs.append(val_acc.item())


    print("Epoch: {}/{}...".format(e+1, epochs),
          "Loss: {:.3f}...".format(np.mean(losses)),
                  "Val Loss: {:.3f}...".format(np.mean(val_losses)),
                  "Accuracy: {:.3f}...".format(np.mean(train_acc)),
                  "Val Accuracy: {:.3f}%".format(np.mean(val_accs)*100))
    if (np.mean(val_losses))<best_val_loss:
        best_val_loss = np.mean(val_losses)
        torch.save(model, MODEL_PATH+'SentimentClf_model.pt')

Epoch: 1/1... Loss: 0.324... Val Loss: 0.340... Accuracy: 0.865... Val Accuracy: 86.500%


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type SentimentClassification. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [0]:
def test(model, test_iter, criterion):
    test_losses = []
    test_accs = []
    model.eval()
    for i_test, batch_test in enumerate(test_iter):

        test_pred = model(batch_test.clean_review[0], batch_test.clean_review[1]).squeeze(1)
        test_loss = criterion(test_pred, batch_test.sentiment.float())
        test_acc = calculateAccuracy(test_pred, batch_test.sentiment)
        test_losses.append(test_loss.item())
        test_accs.append(test_acc.item())

    return np.mean(test_losses), np.mean(test_accs)

In [0]:
model.load_state_dict(torch.load('Sentiment_model.pt'))

test_loss, test_acc = test(model, test_iter, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

In [19]:
import spacy
nlp = spacy.load('en')
model_ = torch.load(MODEL_PATH+'SentimentClf_model.pt')
model_.eval()

SentimentClassification(
  (embedding): Embedding(141208, 100, padding_idx=1)
  (lstm): LSTM(100, 256, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=512, out_features=1, bias=True)
  (sig): Sigmoid()
)

In [0]:
def predict_sentiment(model_, sentence):
    model_.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

In [21]:
predict_sentiment(model_, "This film is terrible")

0.3924224078655243

In [22]:
predict_sentiment(model, "This film is awesome")

0.9593377709388733